In [1]:
state = 'WV' 
year = 2010
obj_type = 'average_Polsby_Popper'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of WV has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from WV_plans_2010 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 0 plans from file.


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Running epsilon constraint method for state WV, objective average_Polsby_Popper
####################################################################################################

L = 611489 and U = 623841

****************************************
Trying deviation = 6176.6466666666665
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 611489 and U = 623841
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - W

H23817  3055                       0.2404893    0.27883  15.9%   149  137s
H24451  2949                       0.2404909    0.27493  14.3%   148  139s
 25493  2915 infeasible   37         0.24049    0.27031  12.4%   147  143s
H25921  2915                       0.2405011    0.27031  12.4%   146  143s
 26502  2743 infeasible   39         0.24050    0.26803  11.4%   145  146s
H26505  2743                       0.2405049    0.26803  11.4%   145  146s
H27071  2743                       0.2405113    0.26739  11.2%   145  146s
H27542  2523                       0.2405116    0.26535  10.3%   144  149s
H27542  2523                       0.2405119    0.26535  10.3%   144  149s
H27542  2523                       0.2405138    0.26535  10.3%   144  150s
H27664  2523                       0.2405151    0.26535  10.3%   144  150s
H27989  2527                       0.2405154    0.26509  10.2%   143  150s
H28106  2527                       0.2405157    0.26509  10.2%   143  150s
 30171  1981    0.24931  

 64385  6938 infeasible   52         0.23631    0.26814  13.5%   139  111s
 69834  6985     cutoff   52         0.23631    0.26511  12.2%   135  115s
 76241  7457    0.25096   48  257    0.23631    0.25507  7.94%   130  122s
 80185  7919    0.23921   48  142    0.23631    0.25000  5.79%   127  126s
 87938  8196    0.24766   46  260    0.23631    0.24920  5.46%   122  132s
 93214  8103    0.23768   53  186    0.23631    0.24875  5.27%   119  135s
 98321  7684    0.23891   45  185    0.23631    0.24837  5.10%   117  140s
 104371  6659 infeasible   56         0.23631    0.24772  4.83%   115  146s
 108645  5999 infeasible   50         0.23631    0.24732  4.66%   113  150s
 112722  4991     cutoff   43         0.23631    0.24654  4.33%   112  155s
 118014  4919    0.24310   53  237    0.23631    0.24479  3.59%   110  161s
 123968  4345 infeasible   57         0.23631    0.24426  3.37%   108  165s
 127987  4470 infeasible   72         0.23631    0.24219  2.49%   106  170s
 137427  4161    0.

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
no_solution_region = [0, min(round(r[2],1) for r in result)]
print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)